Collect imbalanced reactions from all models without MEMOTE

Imports

In [3]:
import cobra
from cobra.io import read_sbml_model
from cobra import Reaction
import os
import pandas as pd
from cobra.util.solver import solvers

Paths

In [4]:
model_path = '/home/frowin/Desktop/master_thesis/MCSM/models/root_models_all_modified/'

save_path = '/home/frowin/Desktop/master_thesis/notebooks/05_EEGCs/Involved_reactions_roots/'

test_models = ["modified_Root4.xml", "modified_Root9.xml", "modified_Root11.xml", "modified_Root16D2.xml", "modified_Root22.xml"]

Set Solver

In [5]:
cobra.Configuration().solver = "cplex"

All metabolites to be tested for EEGCs

In [6]:
energy_metabolites = ['atp_c', 'ctp_c', 'gtp_c', 'utp_c', 'itp_c', 'nadph_c', 'nadh_c', 'fadh2_c', 'fmnh2_c', 'q8h2_c', 'mql8_c', 'mql6_c', 'mql7_c', '2dmmql8_c', 'accoa_c', 'glu__L_c']

Energy couples for all tested metabolites as dict

In [7]:
ENERGY_COUPLES = {
    "atp_c": "adp_c",
    "ctp_c": "cdp_c",
    "gtp_c": "gdp_c",
    "utp_c": "udp_c",
    "itp_c": "idp_c",
    "nadph_c": "nadp_c",
    'nadh_c': 'nad_c',
    "fadh2_c": "fad_c",
    "fmnh2_c": "fmn_c",
    "q8h2_c": "q8_c",
    "mql8_c": "mqn8_c",
    "mql6_c": "mqn6_c",
    "mql7_c": "mqn7_c",
    "2dmmql8_c": "2dmmq8_c",
    "accoa_c": "coa_c",
    "glu__L_c": "akg_c"
}

Functions

In [8]:
def close_boundaries_sensibly(model):
    """
    Return a cobra model with all boundaries closed and changed constraints.

    In the returned model previously fixed reactions are no longer constrained
    as such. Instead reactions are constrained according to their
    reversibility. This is to prevent the FBA from becoming infeasible when
    trying to solve a model with closed exchanges and one fixed reaction.

    Parameters
    ----------
    model : cobra.Model
        The metabolic model under investigation.

    Returns
    -------
    cobra.Model
        A cobra model with all boundary reactions closed and the constraints
        of each reaction set according to their reversibility.
    """
    
    for rxn in model.reactions:
        if rxn.reversibility:
            rxn.bounds = -1, 1
        else:
            rxn.bounds = 0, 1
    for boundary in model.boundary:
        boundary.bounds = (0, 0)
    return model

In [9]:
def detect_energy_generating_cycles(model, metabolite_id):

    dissipation_rxn = Reaction("Dissipation")
    if metabolite_id in ["atp_c", "ctp_c", "gtp_c", "utp_c", "itp_c"]:
        # build nucleotide-type dissipation reaction
        dissipation_rxn.add_metabolites(
            {
                model.metabolites.get_by_id('h2o_c'): -1,
                model.metabolites.get_by_id('h_c'): 1,
                model.metabolites.get_by_id('pi_c'): 1,
            }
        )
    elif metabolite_id in ["nadph_c", "nadh_c"]:
        # build nicotinamide-type dissipation reaction
        dissipation_rxn.add_metabolites(
            {model.metabolites.get_by_id('h_c'): 1}
        )
    elif metabolite_id in [
        "fadh2_c",
        "fmnh2_c",
        "q8h2_c",
        "mql8_c",
        "mql6_c",
        "mql7_c",
        "2dmmql8_c"
    ]:
        # build redox-partner-type dissipation reaction
        dissipation_rxn.add_metabolites(
            {model.metabolites.get_by_id('h_c'): 2}
        )
    elif metabolite_id == "accoa_c":
        dissipation_rxn.add_metabolites(
            {
                model.metabolites.get_by_id('h2o_c'): -1,
                model.metabolites.get_by_id('h_c'): 1,
                model.metabolites.get_by_id('ac_c'): 1
            }
        )
    elif metabolite_id == "glu__L_c":
        dissipation_rxn.add_metabolites(
            {
                model.metabolites.get_by_id('h2o_c'): -1,
                model.metabolites.get_by_id('h_c'): 2,
                model.metabolites.get_by_id('nh4_c'): 1
            }
        )

    dissipation_product = ENERGY_COUPLES[metabolite_id]
    dissipation_rxn.add_metabolites({model.metabolites.get_by_id(metabolite_id): -1, model.metabolites.get_by_id(dissipation_product): 1})
    model = close_boundaries_sensibly(model)
    model.add_reactions([dissipation_rxn])
    print(model.reactions.get_by_id("Dissipation").reaction)
    print(model.reactions.get_by_id("Dissipation").bounds)
    model.objective = dissipation_rxn
    print(model.objective.expression)
    solution = model.optimize(raise_error=True)
    print(solution)
    if solution.objective_value > 0.0:
        return (
            solution.fluxes[solution.fluxes.abs() != 0.0]
            .index.drop(["Dissipation"])
            .tolist()
        )
    else:
        return []

In [10]:
test_model = ['modified_Root275.xml']

In [15]:
EGCs = {}
for m in test_model:
    print(m)
    model = read_sbml_model(model_path + f'{m}')
    EGCs[m] = {}
    for metabolite in energy_metabolites:
        EGCs[m][metabolite] = []
        with model:
            print(f"Checking metabolite: {metabolite}")
            try:
                result = detect_energy_generating_cycles(model, metabolite)
            except KeyError:
                print(f"Metabolite {metabolite} not found in model.")
                result = f"Metabolite {metabolite} not found in model."
                continue
            if result:
                EGCs[m][metabolite] = result

modified_Root275.xml
Checking metabolite: atp_c
atp_c + h2o_c --> adp_c + h_c + pi_c
(0.0, 1000.0)
1.0*Dissipation - 1.0*Dissipation_reverse_fc09b
<Solution 1.000 at 0x7fd478a82b60>
Checking metabolite: ctp_c
ctp_c + h2o_c --> cdp_c + h_c + pi_c
(0.0, 1000.0)
1.0*Dissipation - 1.0*Dissipation_reverse_fc09b
<Solution 1.000 at 0x7fd478a81180>
Checking metabolite: gtp_c
gtp_c + h2o_c --> gdp_c + h_c + pi_c
(0.0, 1000.0)
1.0*Dissipation - 1.0*Dissipation_reverse_fc09b
<Solution 1.000 at 0x7fd478a68ca0>
Checking metabolite: utp_c
h2o_c + utp_c --> h_c + pi_c + udp_c
(0.0, 1000.0)
1.0*Dissipation - 1.0*Dissipation_reverse_fc09b
<Solution 1.000 at 0x7fd478a6a680>
Checking metabolite: itp_c
h2o_c + itp_c --> h_c + idp_c + pi_c
(0.0, 1000.0)
1.0*Dissipation - 1.0*Dissipation_reverse_fc09b
<Solution 1.000 at 0x7fd478a6a5f0>
Checking metabolite: nadph_c
nadph_c --> h_c + nadp_c
(0.0, 1000.0)
1.0*Dissipation - 1.0*Dissipation_reverse_fc09b
<Solution 1.200 at 0x7fd478a69090>
Checking metabolite: na

In [16]:
EGCs

{'modified_Root275.xml': {'atp_c': ['ALAR',
   'ATPS4rpp',
   'CAT',
   'CYOO2pp',
   'CYTMQOR3pp',
   'DAAD',
   'G3PD',
   'G3PD6',
   'PDYXPT_c',
   'PYDXO'],
  'ctp_c': ['ALAR',
   'ATPS4rpp',
   'CAT',
   'CYOO2pp',
   'CYTMQOR3pp',
   'DAAD',
   'G3PD',
   'G3PD6',
   'NDPK3',
   'PDYXPT_c',
   'PYDXO'],
  'gtp_c': ['ALAR',
   'ATPS4rpp',
   'CAT',
   'CYOO2pp',
   'CYTMQOR3pp',
   'DAAD',
   'G3PD',
   'G3PD1ir',
   'G3PD6',
   'NADHPO',
   'NDPK1',
   'PDYXPT_c',
   'PYDXO'],
  'utp_c': ['ALAR',
   'ATPS4rpp',
   'CAT',
   'CYOO2pp',
   'CYTMQOR3pp',
   'DAAD',
   'G3PD',
   'G3PD1ir',
   'G3PD6',
   'NADHPO',
   'NDPK2',
   'PDYXPT_c',
   'PYDXO'],
  'itp_c': ['ALAR',
   'ATPS4rpp',
   'CAT',
   'CYOO2pp',
   'CYTMQOR3pp',
   'DAAD',
   'G3PD',
   'G3PD1ir',
   'G3PD6',
   'NADHPO',
   'NDPK9',
   'PDYXPT_c',
   'PYDXO'],
  'nadph_c': ['AACPS3',
   'AADa',
   'AADb',
   'ACOAD7',
   'AGPAT160',
   'ALATA_L',
   'ATPS4rpp',
   'CAT',
   'CTECOAI7',
   'CYOO2pp',
   'CYTMQOR3pp'

In [17]:
for key, value in EGCs['modified_Root275.xml'].items():
    print(f'{key}: {len(value)}')

atp_c: 10
ctp_c: 11
gtp_c: 13
utp_c: 13
itp_c: 13
nadph_c: 32
nadh_c: 32
fadh2_c: 41
fmnh2_c: 41
q8h2_c: 39
mql8_c: 38
mql6_c: 0
mql7_c: 27
2dmmql8_c: 40
accoa_c: 24
glu__L_c: 48


In [14]:
EGCs_df = pd.DataFrame.from_dict(EGCs)
EGCs_df.to_csv(save_path + '/EEGCs_roots.csv')
print(EGCs_df)

                                        modified_Root275.xml
2dmmql8_c  [AACPS3, AADa, AADb, ACACT5r, ACACT5r_1, ACACT...
accoa_c    [ACACT5r, ACACT5r_1, ACCOAL, ACOAD4_1, ACOAD4f...
atp_c      [ALAR, ATPS4rpp, CAT, CYOO2pp, CYTMQOR3pp, DLY...
ctp_c      [ACCOAL, ALAR, APAT_1, ATPS4rpp, CAT, CPK1, CS...
fadh2_c    [1P2CBXLR, AACPS3, AADa, AADb, ACACT5r, ACACT5...
fmnh2_c    [1P2CBXLR, AACPS3, AADa, AADb, ACACT5r, ACACT5...
glu__L_c   [AACPS3, AADa, AADb, ACACCT, ACACT5r, ACACT5r_...
gtp_c      [ALAR, ATPS4rpp, CAT, CYOO2pp, CYTMQOR3pp, DLY...
itp_c      [ADK1, ADK4, ALAR, ATPS4rpp, CAT, CYOO2pp, CYT...
mql6_c                                                    []
mql7_c     [2AGPE160tipp, 2AGPEAT160, ACACT5r, ACACT5r_1,...
mql8_c     [1P2CBXLR, AACPS3, AADa, AADb, ACACT5r, ACACT5...
nadh_c     [1P2CBXLR, AACPS3, AADa, AADb, ACACT5r, ACACT5...
nadph_c    [AACPS3, AADa, AADb, ACACT5r, ACACT5r_1, ACCOA...
q8h2_c     [1P2CBXLR, AACPS3, AADa, AADb, ACACT5r, ACACT5...
utp_c      [ADK1, ALAR, 